# Monte Carlo World Cup Simulation

### Elo Data
###### Source date: 05/22/2018 
https://www.eloratings.net/

In [2]:
participant_elos = {'Australia':1714, 
                    'Iran':1796,
                    'Japan':1692, 
                    'Saudi Arabia': 1600,
                    'South Korea':1745, 
                    'Egypt':1646,
                    'Morocco':1711,
                    'Nigeria':1699,
                    'Senegal':1747,
                    'Tunisia':1649,
                    'Costa Rica':1745,
                    'Mexico':1859,
                    'Panama':1669,
                    'Argentina': 1985,
                    'Brazil':2131,
                    'Colombia':1935,
                    'Peru':1906,
                    'Uruguay':1891,
                    'Belgium':1931,
                    'Croatia':1853,
                    'Denmark':1843,
                    'England':1941,
                    'France':1984,
                    'Germany':2092,
                    'Iceland':1787,
                    'Poland':1831,
                    'Portugal':1975,
                    'Russia':1685,       # Host Country
                    'Serbia':1770,
                    'Spain':2049,
                    'Sweden':1796,
                    'Switzerland':1879              
                   }

In [3]:
groups = {'Group_A': ['Russia', 'Saudi Arabia', 'Egypt', 'Uruguay'],
          'Group_B': ['Portugal', 'Spain', 'Morocco', 'Iran'],
          'Group_C': ['France', 'Australia', 'Peru', 'Denmark'],
          'Group_D': ['Argentina', 'Iceland', 'Croatia', 'Nigeria'],
          'Group_E': ['Brazil', 'Switzerland', 'Costa Rica', 'Serbia'],
          'Group_F': ['Germany', 'Mexico', 'Sweden', 'South Korea'],
          'Group_G': ['Belgium', 'Panama', 'Tunisia', 'England'],
          'Group_H': ['Poland', 'Senegal', 'Colombia', 'Japan']       
        }

In [4]:
for g in groups:
    for t in groups[g]:
        print(participant_elos[t])

1685
1600
1646
1891
1975
2049
1711
1796
1984
1714
1906
1843
1985
1787
1853
1699
2131
1879
1745
1770
2092
1859
1796
1745
1931
1669
1649
1941
1831
1747
1935
1692


#### Win probability formula
$$W_e = \frac{1}{10^{\frac{-d_r}{400}} + 1}$$


In [49]:
from itertools import combinations

def win_probability(team, opponent, elo):
    team_elo = elo[team]
    opponent_elo = elo[opponent]
    return (1 / (10 ** ( - ( team_elo-opponent_elo) / 400) + 1 ) )

def single_group_run(group_name):
    group_members = groups[group_name]
    for game in combinations(group_members, 2):
        print(game, win_probability(game[0], game[1], participant_elos))

def simulate_single_group_winners(group_name, number_of_runs):
    group_members = groups[group_name]
    for game in combinations(group_members, 2):
        print(game, win_probability(game[0], game[1], participant_elos))
        
simulate_single_group_winners('Group_A',10)

('Russia', 'Saudi Arabia') 0.6199413590445233
('Russia', 'Egypt') 0.5558909611168531
('Russia', 'Uruguay') 0.2340053292335112
('Saudi Arabia', 'Egypt') 0.43418479693076684
('Saudi Arabia', 'Uruguay') 0.15774132752957445
('Egypt', 'Uruguay') 0.19618148057531573


In [40]:
from itertools import combinations
list(combinations([1,2,3,4],2))

[(1, 2), (1, 3), (1, 4), (2, 3), (2, 4), (3, 4)]